<a href="https://colab.research.google.com/github/ProfDoeg/Colegio_Invisible/blob/master/06_cuaderno.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install eciespy eth_keys cryptos qrcode base58 
! git clone https://github.com/ProfDoeg/Colegio_Invisible.git

In [ ]:
import hashlib
import os
import getpass
import ecies
import eth_keys
import cryptos
import qrcode
import base58
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def import_privKey(privkey_filepath,password=None):
    if password==None:
        password = getpass.getpass("Input password for decrypting keyfile: ")
    password=password.encode()
    f=open(privkey_filepath,'rb')
    decrypted_bytes=ecies.aes_decrypt(key=hashlib.sha256(password).digest(),cipher_text=f.read())
    privKey=eth_keys.keys.PrivateKey(decrypted_bytes)
    return privKey

def import_pubKey(pubkey_filepath):
    f=open(pubkey_filepath,'rb')
    privKey=eth_keys.keys.PublicKey(f.read())
    return privKey

def import_addr(addr_filepath):
    return open(addr_filepath,'rb').read().decode()

# starting from existing keys

## Empezar desde Keys en Google Drive
## (Start from Keys on Google Drive)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
! cp drive/MyDrive/cinv/llaves.zip llaves.zip 
! unzip llaves.zip

## Comenzar desde las Claves en la Disco Local
## (Start from Keys on Local Drive)

In [ ]:
from google.colab import files
fu=files.upload();

In [ ]:
! unzip llaves.zip

In [ ]:
! ls llaves

In [ ]:
! ls
#! rm -r llaves
#! rm llaves_.zip

# Import Private Key and Address

In [ ]:
privKey=import_privKey('llaves/mi_prv.enc','')
addr=import_addr('llaves/mi_addr.bin')

In [ ]:
addr

In [ ]:
from IPython.display import Image
qr=Image('llaves/mi_addr.png')
display(qr)

# Decrypt Credentials

In [ ]:
! python Colegio_Invisible/scripts/aes_decrypt.py Colegio_Invisible/secrets/client_secret.aes Colegio_Invisible/secrets/client_secret.json 

# Authenticate and Connect to Sheets

In [ ]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
# use creds to create a client to interact with the Google Drive API
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name('Colegio_Invisible/secrets/client_secret.json', scope)
client = gspread.authorize(creds)
sheets = client.open("signatures")

# View Sheets

In [ ]:
sheets.worksheets()

In [ ]:
sheet4=sheets.worksheets()[4]
sheet4.row_values(1)

In [ ]:
#sheet4.clear()
#sheet4.append_row(["name", "address"])

# Input Name

In [ ]:
my_name=input('input name')

# Posted Names

In [ ]:
peeps=sheet4.col_values(1)[1:]
peeps

If your name is in the list then write address to the appropriate cell of sheet. 

If name is not present append name and address to sheet

In [ ]:
if not (my_name in peeps) :
  sheet4.append_row([my_name,addr])
else:
  sheet4.update_cell(row=peeps.index(my_name)+2,col=2,value=addr)

# View Sheet

In [ ]:
import pandas as pd

dataframe4 = pd.DataFrame(sheet4.get_all_records())
dataframe4

# Extract Addresses

In [ ]:
addresses=dataframe4.address.tolist()
addresses

import cryptos library and create Dogecoin object

In [ ]:
import cryptos

In [ ]:
doge=cryptos.Doge()

# Identify Funding Address as `banco`

In [ ]:
banco=addresses[0]
banco

## Determine Unspent Outputs

In [ ]:
unspents=doge.unspent(banco)
unspents

## Sum Unspent Outputs

In [ ]:
unspent_total=sum([unspent['value'] for unspent in unspents])
unspent_total

In [ ]:
gift=10_00_000_000
gift_total = gift * len(addresses)
mining_fee = 2_00_000_000
change = unspent_total - gift_total - mining_fee 

In [ ]:
outputs=[{'value': gift, 'address': address} for address in addresses]
outputs.append({'value': change, 'address': banco})
outputs

# Make Funding Transaction

In [ ]:
fund_txn=doge.mktx(unspents,outputs )
fund_txn

#Sign Transaction

In [ ]:
doge.signall(fund_txn,privKey.to_hex()[2:])

In [ ]:
fund_txn

In [ ]:
fund_txn_hex=cryptos.serialize(fund_txn)
fund_txn_hex

# Check kilobyte size of transaction

In [ ]:
kb=len(fund_txn_hex)/2000
kb

In [ ]:
mining_fee/1E8/kb

# Broadcast Transaction

In [ ]:
broadcast_fund_txn=doge.pushtx(fund_txn_hex)
broadcast_fund_txn

# Inspect Transaction

In [ ]:
inspect_fund_txn=doge.fetchtx( broadcast_fund_txn_hex['data']['txid'])
inspect_fund_txn